In [31]:
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers import default_data_collator
from transformers import Wav2Vec2FeatureExtractor, HubertForSequenceClassification
import torch
import torch.nn as nn
from datasets import Dataset, Audio, Value, Features,load_dataset,ClassLabel
from transformers import Wav2Vec2Processor
from transformers.modeling_outputs import SequenceClassifierOutput
import numpy as np
from transformers import AdamW,get_scheduler
from datasets import load_metric
from tqdm.auto import tqdm
import os
from torch.utils.data import DataLoader
x = [str(i) for i in range(0,100,1)]
features = Features(
    {
        "id": Value("string"),
        "speaker_id": Value("string"), 
        'path': Value('string'),
        "audio": Audio(sampling_rate=16000),
        "label": ClassLabel(num_classes=100,names=x,names_file=None,id=None),
    }
)


dataset = load_dataset('csv', data_files={'train': 'train_100.csv', 'dev': 'dev_100.csv', 'test': 'test_100.csv',},features=features)
dataset = dataset['test']
dataset = dataset.sort("label")
sampling_rate = dataset.features["audio"].sampling_rate
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/hubert-base-superb-sid")
#feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/wav2vec2-base-superb-sid")

Using custom data configuration default-50c0c0358139fbe1
Found cached dataset csv (/storage/home/hcocice1/vprakash40/.cache/huggingface/datasets/csv/default-50c0c0358139fbe1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|██████████| 3/3 [00:00<00:00, 186.13it/s]
Loading cached sorted indices for dataset at /storage/home/hcocice1/vprakash40/.cache/huggingface/datasets/csv/default-50c0c0358139fbe1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-d89fbf7cda23da10.arrow


In [32]:
max_duration = 10.0  # seconds

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

def preprocess_function(examples):
    audio_arrays = [examples['audio'][i]['array'] for i in range(len(examples['id']))]
#     audio_arrays = examples["audio"]['array']
#     audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True,
        padding=True

    )
    return inputs

In [33]:
dataset = dataset.map(preprocess_function, remove_columns=["audio"], batched=True,num_proc=24,batch_size=32)
dataset

Loading cached processed dataset at /storage/home/hcocice1/vprakash40/.cache/huggingface/datasets/csv/default-50c0c0358139fbe1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-2af97893ac7c8ce8.arrow
Loading cached processed dataset at /storage/home/hcocice1/vprakash40/.cache/huggingface/datasets/csv/default-50c0c0358139fbe1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-e4dfdb2ae8e3f15f.arrow
Loading cached processed dataset at /storage/home/hcocice1/vprakash40/.cache/huggingface/datasets/csv/default-50c0c0358139fbe1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-4b0784f67d2b4f03.arrow
Loading cached processed dataset at /storage/home/hcocice1/vprakash40/.cache/huggingface/datasets/csv/default-50c0c0358139fbe1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-c710592794e2fbac.arrow
Loading cached processed dataset at /storage/home/hcocice1/vprakash40/.cache/huggingface/dat

Dataset({
    features: ['id', 'speaker_id', 'path', 'label', 'input_values', 'attention_mask'],
    num_rows: 632
})

In [34]:
dataset.set_format("torch",columns=["id","input_values","attention_mask", "label"])
# train_data_collator = default_data_collator
# dev_data_collator = default_data_collator
# test_data_collator = default_data_collator

# train_dataloader = DataLoader(
#     dataset["train"], shuffle=True, batch_size=32, collate_fn=train_data_collator
# )
# dev_dataloader = DataLoader(
#     dataset["dev"], batch_size=32, collate_fn=dev_data_collator
# )

In [35]:
dataset

Dataset({
    features: ['id', 'speaker_id', 'path', 'label', 'input_values', 'attention_mask'],
    num_rows: 632
})

In [36]:
from transformers import Wav2Vec2ForSequenceClassification
model_ft = HubertForSequenceClassification.from_pretrained("superb/hubert-base-superb-sid")
#model_ft = Wav2Vec2ForSequenceClassification.from_pretrained("superb/wav2vec2-base-superb-sid")

# model_ft = Wav2Vec2ForSequenceClassification.from_pretrained("superb/wav2vec2-base-superb-sid")
# feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/wav2vec2-base-superb-sid")


In [37]:
model_ft.eval()

HubertForSequenceClassification(
  (hubert): HubertModel(
    (feature_extractor): HubertFeatureEncoder(
      (conv_layers): ModuleList(
        (0): HubertGroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (2): HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (3): HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (4): HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False

In [38]:
dataset

Dataset({
    features: ['id', 'speaker_id', 'path', 'label', 'input_values', 'attention_mask'],
    num_rows: 632
})

In [55]:
metric_name="accuracy"
metric = load_metric(metric_name)
model_ft.eval()


#dataset.set_format("torch",columns=["id","input_values", "attention_mask", "label"])


test_dataloader = DataLoader(
    dataset, batch_size=32, collate_fn=default_data_collator
)

predictions_list = []
logits_list = []
for batch in test_dataloader:
#     print(batch.items())
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model_ft(**batch)

    logits = outputs.logits
    logits_list.append(logits)
    predictions = torch.argmax(logits, dim=-1)
    predictions_list = [y for x in [predictions_list, predictions] for y in x]
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8275316455696202}

In [54]:
id1 = dataset['id']
logits
predictions_list
pred = dict(zip(id1, predictions_list))
pred

torch.Size([24, 1251])

In [22]:
import torch
import librosa
from datasets import load_dataset
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor

def map_to_array(example):
    speech, _ = librosa.load(example["file"], sr=16000, mono=True)
    example["speech"] = speech
    return example

# load a demo dataset and read audio files
dataset = load_dataset("anton-l/superb_demo", "si", split="test")
dataset = dataset.map(map_to_array)

model = Wav2Vec2ForSequenceClassification.from_pretrained("superb/wav2vec2-base-superb-sid")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/hubert-base-superb-sid")

# compute attention masks and normalize the waveform if needed
inputs = feature_extractor(dataset[:2]["speech"], sampling_rate=16000, padding=True, return_tensors="pt")

Found cached dataset superb_demo (/storage/home/hcocice1/vprakash40/.cache/huggingface/datasets/anton-l___superb_demo/si/1.9.0/77d23894ff429329a7fe80f9007cabb0deec321316f8dda1a1e9d10ffa089d08)
Loading cached processed dataset at /storage/home/hcocice1/vprakash40/.cache/huggingface/datasets/anton-l___superb_demo/si/1.9.0/77d23894ff429329a7fe80f9007cabb0deec321316f8dda1a1e9d10ffa089d08/cache-68bfe8a618136781.arrow
/storage/home/hcocice1/vprakash40/.conda/envs/ece6254/lib/python3.8/site-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [23]:
inputs

{'input_values': tensor([[ 0.0290,  0.0117,  0.0058,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0226, -0.0217, -0.0222,  ..., -0.0038, -0.0030, -0.0024]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)}